In [1]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

#
# data
#
from techminer import *


Records(directory="../data/processed/debug").import_records(source="../data/tmp/debug-scopus.csv", filetype="scopus")


# wos_df = load_file(filepath="techminer/tests/data/savedrecs.txt", filetype="wos", datastorepath="./datastore")
# dim_df = load_file(filepath="techminer/tests/data/dimensions.csv", filetype="dimensions", datastorepath="./datastore")

2021-10-30 15:57:24 - INFO - Reading file '../data/tmp/debug-scopus.csv' ...
2021-10-30 15:57:25 - INFO - Formatting dataset ...
2021-10-30 15:57:33 - INFO - Formating authors ...
2021-10-30 15:57:33 - INFO - Formating authors_id ...
2021-10-30 15:57:33 - INFO - Formatting columns ...
2021-10-30 15:57:33 - INFO - Computing new columns ...
2021-10-30 15:57:34 - INFO - Dropping duplicates ...
2021-10-30 15:57:34 - INFO - Transforming British to American ...


100%|██████████| 1737/1737 [02:09<00:00, 13.38it/s]


2021-10-30 15:59:45 - INFO - Records saved/merged to '../data/processed/debug/records.csv'
2021-10-30 15:59:45 - INFO - The file '../data/tmp/debug-scopus.csv' was successfully imported.


In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.data import *

# coverage(directory_or_records="../data/processed/debug")
describe_records(directory_or_records="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

#
# features
#
from techminer.features import *

# extract_coutries(directory_or_records="../data/processed/debug", column='affiliations')






In [2]:
#
# query modules
#
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.query import * 
from techminer.visualization import worldmap



# term_analysis(directory_or_records="../data/processed/debug", column='authors')
### count_terms_by_column(directory_or_records="../data/processed/debug", sep="; ", column='authors')
# time_analysis(directory_or_records="../data/processed/debug")
### count_local_citations_by_year(directory_or_records="../data/processed/debug")
### count_global_citations_by_year(directory_or_records="../data/processed/debug")
### count_documents_by_year(directory_or_records="../data/processed/debug")
### mean_global_citations_by_year(directory_or_records="../data/processed/debug")
# most_cited_documents(directory_or_records="../data/processed/debug")
# compute_impact_index(directory_or_records="../data/processed/debug", column='authors')




# ---
# worldmap(count_documents_by_term(directory_or_records="../data/processed/debug", column='countries'))



/usr/local/lib/python3.7/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,h_index,g_index,num_documents,global_citations,m_index,global_citations_per_year,avg_global_citations
Abad JD,1,1,1,6,0.25,1.50,6.0
Abi-Mansour A,1,1,1,1,0.33,0.33,1.0
Abraham MJ,1,1,1,4746,0.14,678.00,4746.0
Abreu M,1,1,1,4,0.50,2.00,4.0
Aggoun A,1,1,1,1,0.33,0.33,1.0
...,...,...,...,...,...,...,...
van den Oord G,0,0,2,0,0.00,0.00,0.0
von Reis E,0,0,1,0,0.00,0.00,0.0
Álvarez-Martínez D,0,0,1,0,0.00,0.00,0.0
Çalışkan M,0,0,1,0,0.00,0.00,0.0


In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import Records

Records(directory=)

# core_sources(directory_or_records="../data/processed/debug")
# core_authors(directory_or_records="../data/processed/debug")
# lotka(core_authors(directory_or_records="../data/processed/debug"), colormap='Blues', figsize=(6,3))


In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from techminer.visualization import bradford

bradford(directory_or_records="../data/processed/debug")

In [ ]:
import pandas as pd

from techminer.data.records import load_records

# sorted(load_records(directory="../data/processed/debug").columns)
load_records(directory="../data/processed/debug")

In [ ]:
from src.features.apply_keywords_thesaurus import apply_keywords_thesaurus
from src.features.apply_institutions_thesaurus import apply_institutions_thesaurus

# Ok!
apply_keywords_thesaurus(datastoredir="../data/processed/debug")
apply_institutions_thesaurus(datastoredir="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.analysis import Bradford

bradford = Bradford(datastoredir="../data/processed/debug")

bradford.plot(max_items=8, cmap='Blues', figsize=(8, 5), fontsize=9)

In [ ]:
bradford.sources_

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from src.analysis import TopDocuments

TopDocuments(datastoredir="../data/processed/debug").detailed_

In [ ]:
from techminer.analysis.top_documents import TopDocuments

TopDocuments(datastorepath="./datastore").print()

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").plot(figsize=(8, 4))

In [ ]:
from techminer.analysis.core_sources import CoreSources

CoreSources(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.coverage import Coverage

Coverage(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.worldmap import WorldMap

WorldMap(datastorepath="./datastore").plot(figsize=(9, 5), colormap='Blues')

In [ ]:
import pandas as pd

pd.read_csv("./datastore/datastore.csv").authors_id